In [5]:
import numpy as np
from tqdm.notebook import tqdm
import trimesh
from queue import PriorityQueue
import matplotlib.pyplot as plt
from make_vtk import write_lines_to_vtk,write_points_to_vtk
from vtk_revise import read_vtk,write_vtk
from faster_generate_cone import generate_vectors_in_cone,plot_vectors_in_3d
# import cupy as cp

In [6]:
white = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.white.vtk')
pial = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.pial.vtk')

white_vertices = white['vertices']
white_faces = white['faces'][:,1:]

pial_vertices = pial['vertices']
pial_faces = pial['faces'][:,1:]

In [7]:
# Load your target mesh
origin_mesh = trimesh.Trimesh(vertices=white_vertices, faces=white_faces)
target_mesh = trimesh.Trimesh(vertices=pial_vertices, faces=pial_faces)

# origin vertex normals
origin_directions = origin_mesh.vertex_normals.copy()
white2pial_dir = pial_vertices - white_vertices
dir_sign = (np.sum(white2pial_dir*origin_directions,axis=1) < 0)
origin_directions[dir_sign] *= -1

print(white_vertices.shape)

(142422, 3)


In [8]:
# Pre-compute all the cone directions for all origins
all_origins = []
all_directions = []

for idx in tqdm(range(len(white_vertices))):
    origin_pos = white_vertices[idx]
    origin_cone_directions = generate_vectors_in_cone(origin_directions[idx], 15, 50)
    all_origins.extend([origin_pos] * len(origin_cone_directions))
    all_directions.extend(origin_cone_directions)
    


# Find ray-mesh intersections for all origins and directions at once
locations, index_ray, index_tri = target_mesh.ray.intersects_location(ray_origins=all_origins, ray_directions=all_directions)



  0%|          | 0/142422 [00:00<?, ?it/s]

In [18]:
# Now, process the results

threshold = 7

pairs = []
thicknesses = []
not_intersect = []
too_long_pair = []

index_groups = {}
for i, ray_idx in tqdm(enumerate(index_ray), total=len(index_ray)):
    if ray_idx not in index_groups:
        index_groups[ray_idx] = []
    index_groups[ray_idx].append(i)


  0%|          | 0/26470984 [00:00<?, ?it/s]

In [19]:

# 'idx' is a factor to know which original vertex we are looking at. 
# This will change every 50 iterations (since there are 50 samples for each vertex).
for idx in tqdm(range(0, len(all_origins), 50)):

    # real_idx_1 = np.where((index_ray >= idx) & (index_ray < idx+50))
    real_idx_2 = []
    for i in range(idx, idx+50):
        real_idx_2.extend(index_groups.get(i, []))
        
    subset_locations = locations[real_idx_2]

    subset_origin = all_origins[idx]
    
    distances = [np.linalg.norm(loc - subset_origin) for loc in subset_locations]

    if distances:
        min_distance_index = np.argmin(distances)
        closest_location = subset_locations[min_distance_index]
        closest_distance = distances[min_distance_index]
        
        # pairs.append((subset_origin, closest_location))
        # thicknesses.append(closest_distance)
        
        if closest_distance > threshold:
            thicknesses.append(float('nan'))
            too_long_pair.append((subset_origin, closest_location))
            not_intersect.append(subset_origin)
            
        else : 
            pairs.append((subset_origin, closest_location))
            thicknesses.append(closest_distance)
    else:
        thicknesses.append(float('nan'))
        not_intersect.append(subset_origin)


  0%|          | 0/142422 [00:00<?, ?it/s]

In [23]:
thicknesses = np.array(thicknesses)
white['new_thickness'] = thicknesses

print(max(thicknesses))
print(min(thicknesses))

# write_vtk(white,f'./cone_data/subject1_lh_white_with_thres{threshold}.vtk')
# write_lines_to_vtk(pairs, f"./cone_data/pair_line_with_thres{threshold}.vtk")
# write_points_to_vtk(not_intersect,f"./cone_data/not_intersect_point_with_thres{threshold}.vtk",color=(255,0,0))
# write_lines_to_vtk(too_long_pair, f"./cone_data/long_line_{threshold}.vtk")


6.999047361121566
0.00024512719688008134


In [27]:
test = read_vtk('./cone_data/subject1_lh_white_with_thres7.vtk')

# print(a = test['new_thickness'])

a = test['new_thickness']
print(max(a))
print(min(a))

6.9990473611
0.0
